In [1]:
import json, os
import numpy as np

In [2]:
train_dir = '/mnt/ssd1/ImageNet/train'
val_dir = '/mnt/ssd1/ImageNet/val'
npys_dir = './npys'

class_json = './ImageNet_class_index.json'

In [3]:
num2class, class2num = {}, {}

with open(class_json) as json_file:
    json_data = json.load(json_file)
    for num in json_data:
        num2class[int(num)] = json_data[num][-1]
        class2num[json_data[num][-1]] = int(num)

In [4]:
train_imgs = []
train_info = {i:0 for i in range(1000)}

for label in sorted(os.listdir(train_dir)):
    if label.isnumeric():
        label_dir = os.path.join(train_dir, label)
        for img in sorted(os.listdir(label_dir)):
            if img.endswith('.JPEG'):
                img_dir = os.path.join(label_dir, img)
                train_imgs.append(img_dir)
                train_info[int(label)] += 1

In [5]:
train_npys_dir = {}

for file in sorted(os.listdir(npys_dir)):
    if file.endswith('train.npy'):
        _, e, _, _ = file.split('_')
        if e not in train_npys_dir:
            train_npys_dir[e] = []
        
        train_npys_dir[e].append(os.path.join(npys_dir, file))

In [6]:
train_events = {}

for epoch in train_npys_dir:
    print(epoch)
    train_events[epoch] = {}
    for npy_dir in train_npys_dir[epoch]:
        print(npy_dir)
        npy = np.load(npy_dir)
        for i in range(npy.shape[0]):
            img = train_imgs[i]
            if img not in train_events[epoch]:
                train_events[epoch][img] = {'first_learning_event' : [], 'forgetting_event' : []}

            img_event = npy[i]
            img_event = ''.join(map(str, img_event.tolist()))

            if '1' in img_event:
                train_events[epoch][img]['first_learning_event'].append(1 + img_event.index('1'))
            else:
                train_events[epoch][img]['first_learning_event'].append(-1)

            train_events[epoch][img]['forgetting_event'].append(img_event.count('10'))
    print()

040
./npys/ResNet50_040_00_train.npy
./npys/ResNet50_040_01_train.npy
./npys/ResNet50_040_02_train.npy
./npys/ResNet50_040_03_train.npy
./npys/ResNet50_040_04_train.npy
./npys/ResNet50_040_05_train.npy
./npys/ResNet50_040_06_train.npy
./npys/ResNet50_040_07_train.npy

060
./npys/ResNet50_060_00_train.npy
./npys/ResNet50_060_01_train.npy
./npys/ResNet50_060_02_train.npy
./npys/ResNet50_060_03_train.npy
./npys/ResNet50_060_04_train.npy
./npys/ResNet50_060_05_train.npy
./npys/ResNet50_060_06_train.npy
./npys/ResNet50_060_07_train.npy

090
./npys/ResNet50_090_00_train.npy
./npys/ResNet50_090_01_train.npy
./npys/ResNet50_090_02_train.npy
./npys/ResNet50_090_03_train.npy
./npys/ResNet50_090_04_train.npy
./npys/ResNet50_090_05_train.npy
./npys/ResNet50_090_06_train.npy
./npys/ResNet50_090_07_train.npy

120
./npys/ResNet50_120_00_train.npy
./npys/ResNet50_120_01_train.npy
./npys/ResNet50_120_02_train.npy
./npys/ResNet50_120_03_train.npy
./npys/ResNet50_120_04_train.npy
./npys/ResNet50_120_05_t

In [7]:
# train imgs not learned by model

for epoch in train_events:
    print(epoch)
    dic = {num2class[i] : [] for i in range(1000)}
    for img in train_events[epoch]:
        img_label = int(img.split('/')[5])

        for first_learn in train_events[epoch][img]['first_learning_event']:
            if first_learn != -1:
                break
        else:
            dic[num2class[img_label]].append(img)
            
    with open('./not_learned_train_{}.json'.format(epoch), 'w') as j:
        json.dump(dic, j, indent=4)

040
060
090
120
180
270
